# Код для парсинга профилей и постов LinkedIn

In [1]:
import time
import re
import configparser

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

**Загружаем конфиг**

In [2]:
# путь к файлу расширения для Chrome "Доступ к LinkedIn"
EXTENSION_PATH = '1.5_0.crx'

# конфиг
CFG_FILE = 'parser.ini'

"""
файл конфигурации необходимо предварительно создать,
формат файла parser.ini:
[LINKEDIN]
USER_LOGIN = эл_почта_без_кавычек
USER_PASSWORD = пароль
""";

# загружаем данные из конфига
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    logging.error(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')
    quit() # нужен только при переносе кода в .py

**Общие процедуры и функции**

In [3]:
# получить и отобразить информацию профиля
def get_and_print_profile_info(driver, profile_url):
    driver.get(profile_url)        # открываем ссылку profile_url

    # извлекаем код страницы
    src = driver.page_source

    # предеаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')
    soup.prettify()

    # извлекаем HTML содержаший имя и заголовок
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    #print(f'''{intro}''') # вывод содержимого для контроля

    # в случае ошибки попробуйте изменить используемые здесь теги
    name_loc = intro.find("h1")

    # получаем имя
    name = name_loc.get_text().strip()

    # заголовок, обычно тут пишут где работает или специальность
    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # получаем компанию или специальность
    works_at = works_at_loc.get_text().strip()

    print(f'''
    Name --> {name},
    Works At --> {works_at}
    ''')

    # суффикс к url профиля для отображения всех публикаций пользователя
    POSTS_URL_SUFFIX = 'recent-activity/all/'

    time.sleep(0.5)

    # текущий url профиля пользователя
    cur_profile_url = driver.current_url
    print(f'''
    Profile URL --> {cur_profile_url}
    ''')

    # получим все публикации пользователя
    get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)

In [4]:
# прокрутка страницы, для подгрузки динамического контента
def get_scrolled_page(driver, num_scrolls=15, pause_time=1.5):
    """
    Функция прокручивает страницу, загруженную в экземпляр driver,
    num_scrolls раз, с pause_time паузами между прокрутками.
    Возвращает код страницы.
    """
    # получим высоту прокрутки
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(num_scrolls):
        # прокрутка вниз
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # ожидаем загрузку страницы
        time.sleep(pause_time)

        # новая высота прокрутки и сравниваем с последней высотой прокрутки
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return driver.page_source

In [5]:
# получить и отобразить все публикации из профиля
def get_and_print_user_posts(driver, posts_url):
    driver.get(posts_url)

    # получаем код проскроленой страницы
    src = get_scrolled_page(driver)

    # передаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')
    soup.prettify()

    # получаем список постов
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')

    print(f'''
    Number of posts: {len(posts)}
    ''')
    for post_src in posts:
        # блок с текстом поста
#        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})
        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper'})

        if post_text_div is not None:
            # текст поста
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None

        # если пост содержит текст
        if post_text is not None:
            post_text = post_text.get_text().strip()
            print(f'''
            Post text: {post_text}
            ''')

        # число реакций на пост
        reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # если число реакций записано в виде текста, то используем другое имя класса
        if reaction_cnt is None:
            reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})

        if reaction_cnt is not None:
            reaction_cnt = reaction_cnt.get_text().strip()
            print(f'''
            Reactions: {reaction_cnt}
            ''')

    return

**Создаем и запускаем браузер**

In [6]:
# подключаем расширение к драйверу
options = webdriver.ChromeOptions()
options.add_extension(EXTENSION_PATH)

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
options.page_load_strategy = 'eager'

# запускаем Chrome с расширением
driver = webdriver.Chrome(options=options)

**Вход в LinkedIn**

In [7]:
# открываем страницу входа linkedIn, необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(3.5)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")

# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")

# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
# Формат (синтаксис) написания XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

**Тестовый парсинг профиля и публикаций**

In [8]:
# выполним тестовый парсинг
get_and_print_profile_info(driver, 'https://www.linkedin.com/in/alina-peshkur/')


    Name --> Alina Peshkur,
    Works At --> Digital Business Analyst ▪ Mentor for BAs
    

    Profile URL --> https://www.linkedin.com/in/alina-peshkur/
    

    Number of posts: 20
    

            Post text: О моей субличности бизнес-аналитикаПредыстория: Я “вошла в айти” только на софт- и хард скиллах по 1С. В период онбординга зачитывалась статьями analyst.by и искала для себя истинный путь становления бизнес-аналитиком.  Мой мозг игнорировал информацию о книге Карла Вигерса и о профессиональных курсах, но цеплялся за любое упоминание о Шерлоке Холмсе. Поэтому сперва прочитала все книги о Холмсе с непоколебимой верой, что именно Он раскроет секреты и научит ремеслу анализа )) Сейчас у меня новый проект и нахожусь на любимейшем этапе, когда получаю сообщение типа “Алина, нужна помощь по тикету” и просыпается Он. Мой внутренний Шерлок Холмс, которого уже не остановить. Я готовлю кофе, затачиваю простые карандаши, устраиваюсь поудобнее и начинаю расследование! Шаг за шагом вычит

**Парсинг результатов поискового запроса**

In [10]:
# открываем страницу поиска
# поиск по: data scientist
driver.get('https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=1gy')

# список URL профилей пользователей
profile_urls = []

# число страниц для парсинга
NUM_PAGES_TO_PARSE = 5

# собираем URL-адреса профилей со страницы результатов поиска
for i in range(NUM_PAGES_TO_PARSE):
    search_result_links = driver.find_elements(By.CSS_SELECTOR, "div.entity-result__item a.app-aware-link")

    for link in search_result_links:
        href = link.get_attribute("href")
        if 'linkedin.com/in' in href:
            profile_urls.append(href)

    # !!! кнопку [Далее >] не всегда находит, чаще нет чем да, возможно нужно выполнить скролл страницы вниз
    next_button = driver.find_element(By.CLASS_NAME,'artdeco-pagination__button--next')
    next_button.click()
    
    time.sleep(2)

# убираем повторы, оставляем уникальные профили
profile_urls = list(set(profile_urls))

print(f'''
Найдено профилей: {len(profile_urls)}
''')


Найдено профилей: 44



**Парсим посты из профилей**

In [11]:
# парсим посты
for profile_url in profile_urls:
    get_and_print_profile_info(driver, profile_url)
    time.sleep(2)


    Name --> Dmitriy Bushuev,
    Works At --> Data Scientist
    

    Profile URL --> https://www.linkedin.com/in/dmitriy-bushuev/
    

    Number of posts: 0
    

    Name --> Evgeniya Abasheva,
    Works At --> Data Scientist at Tele2 | Python, SQL, ML
    

    Profile URL --> https://www.linkedin.com/in/ezha30/
    

    Number of posts: 6
    

            Post text: I’m happy to share that I’ve obtained a new certification: Critical thinking from Training Institute!
            

            Reactions: 12
            

            Post text: I’m happy to share that I’ve obtained a new certification: Spark Advanced from IBS!
            

            Reactions: 22
            

            Reactions: 14
            

            Reactions: 7
            

    Name --> Roman Puzevich,
    Works At --> Data Analyst | Data Scientist
    

    Profile URL --> https://www.linkedin.com/in/roman-puzevich-899751261/
    

    Number of posts: 0
    

    Name --> Timur Salakhetdinov,


    Name --> Юрий Северинов,
    Works At --> Data Scientist
    

    Profile URL --> https://www.linkedin.com/in/yuriy-severinov/
    

    Number of posts: 2
    

            Post text: Наконец-то получил долгожданный диплом от Практикума и на душе стало немного спокойнее. Конечно в первую очередь играют роль знания и навыки, а не корочки, но есть какое-то внутреннее удовлетворение от очередного пройденного этапа. Я по-прежнему ищу работу в Data Science и буду рад любой помощи, рекомендации и просто лайку, который поможет большему числу людей увидеть эту запись.Мой гитхаб: https://lnkd.in/eiR2zzXP#OpenToWork #junior #datascience #datascientist #ml #dl
            

            Reactions: 53
            

            Post text: Добрый день! Я в активном поиске работы в роли Data Scientist'а и буду очень благодарен за любую поддержку. Если вы знаете, что кто-то ищет джуна, пожалуйста, сообщите мне. Буду признателен и за лайк этого поста, что бы больше людей его увидело!О себе: закан


    Name --> Daniil Lopatkin,
    Works At --> Data Scientist at Acoustic
    

    Profile URL --> https://www.linkedin.com/in/daniil-lopatkin-6262031a9/
    

    Number of posts: 0
    

    Name --> Sergei Suslov,
    Works At --> Data Scientist – Wildberries
    

    Profile URL --> https://www.linkedin.com/in/sergei-suslov-43821771/
    

    Number of posts: 1
    

            Post text: Товарищи! Поздравляю самую прекрасную часть человечества с международным женским днем. Желаю вам по заветам Клары и Розы справедливости и равенства!
            

            Reactions: 1
            

    Name --> Mark Vasiliev,
    Works At --> Data Scientist
    

    Profile URL --> https://www.linkedin.com/in/mrkvsv/
    

    Number of posts: 1
    

            Post text: 🚀 Data Science Interview Questions and Answers 👨‍💻If you’re moving down the path to becoming a data scientist, you must be prepared to impress prospective employers with your knowledge. And to do that you must be able


    Profile URL --> https://www.linkedin.com/in/liliya-alekhina/
    

    Number of posts: 0
    

    Name --> Sergei Dmitriev,
    Works At --> Data Scientist I Machine Learning Engineer
    

    Profile URL --> https://www.linkedin.com/in/sergei-dmitriev-v/
    

    Number of posts: 0
    

    Name --> Vadim Kulikov,
    Works At --> Head of Research And Development в компании KRYPTEN Research and Production Company
    

    Profile URL --> https://www.linkedin.com/in/kulikov-vadim/
    

    Number of posts: 0
    

    Name --> Oleg Strakhov,
    Works At --> Data scientist - CSC Pharmaceuticals
    

    Profile URL --> https://www.linkedin.com/in/oleg-strakhov-ds/
    

    Number of posts: 5
    

            Post text: If you see this post you have a great opportunity to help me develop a ML-model for predicting lung cancer, which will later help doctors detect the disease at an early stage. You only need to take a short anonymous survey, which will take no more than 5 m


    Name --> Dmitry Morozov,
    Works At --> Data Scientist at Beeline Russia
    

    Profile URL --> https://www.linkedin.com/in/dmitry-morozov-0959a11b5/
    

    Number of posts: 0
    

    Name --> Eugene Yumagulov,
    Works At --> Data Scientist
    

    Profile URL --> https://www.linkedin.com/in/eugene-yumagulov-397801220/
    

    Number of posts: 3
    

            Post text: NN&CV
            

            Reactions: 14
            

            Post text: Hi everyone - I am looking for a new role and would appreciate your support. Thank you in advance for any connections, advice, or opportunities you can offer. #OpenToWork
            

            Reactions: 5
            

            Post text: I just earned a skill badge for Python (Programming Language)! Who's next? See how you do on a #LinkedInSkillAssessment.
            

            Reactions: 4
            

    Name --> Daniil Toropov,
    Works At --> Lead Data Scientist
    

    Profile URL --> https:


    Name --> Alexey Artsukevich,
    Works At --> Data Scientist (NLP) at МИА "Россия сегодня"​
    

    Profile URL --> https://www.linkedin.com/in/alexey-artsukevich-50a736177/
    

    Number of posts: 2
    

            Post text: A short story on how I'm trying to take part in AI Journey Contest 2022/AI4Biology (Organized by Sber - the number one it company in Russia, if you mind). The train dataset consists of 256 spectra, however in order to upload the solution, you need to download a 20GB docker image, rewrite the solution code so that it works in this image, send the archive with the code to the server. And what? In the end you get the message "Status: Failed: main.predict: prediction has failed with unknown error, see the logs". Other participants advise trying to upload the same solution several times - it helped them. The support-person from SBER remains silent. It's not AI competition, it's docker competition.
            

            Reactions: 1
            

      

In [ ]:
# закрываем браузер
driver.quit()